# 1. Global sequence alignments

You are given two sequences AGATTAC and AGTTAC. Assume a match score of 1, a gap penalty of 3 and a substitution score of -1. Using these scores, obtain the global alignment of these two sequences manually in the following two steps:

1. Fill in the entries of the F matrix by applying the recurrence relationship for global alignment to these sequences. Please show the back pointers to the matrix entry/entries that give you the maximal score for any entry.

2. Apply the trace back procedure to obtain an optimal alignment. If there are multiple possible alignments, please show all of them along with their traceback paths.

You may take a picture of your alignments and include the image as submission for homework.  Please type the alignments immediately following.

Alignment:

* AG_TTAC
* AGATTAC

# 2. Local sequence alignments

Perform a manual local alignment for the sequences, GAAGAGTATA and AAGCTATACC.   Assume a match score of 1, a gap penalty of 3 and a substitution score of -1.  Complete the following steps:

1. Fill in the entries of the F matrix using the recurrence relationship for the local alignment of these sequences. Show back pointers to matrix entry/entries that give you the maximal score.

2. Apply the trace back procedure to generate a local alignment.

You may take a picture of your alignments and include the image as submission for homework.  Please type the alignments immediately following.

Alignment:
* TATA
* TATA

# 3. Dynamic Programming Implementation

Implement dynamic programming algorithms for global (Needleman-Wunsch) and local (Smith-Waterman) alignment of protein sequences.  The implementation should be on Google App Engine platform and use a web form to submit the alignment calculation.  The form should have the following fields:

* Sequence 1
* Sequence 2
* Select global or local alignment
* Match Score, Mismatch Score, Linear Gap penalty

Once submitted, the solution should show all optimal alignments, the scoring matrix and traceback onscreen.  Type the URL of the application below?

In [39]:
# is gap_pen going to be input as positive or neg (right now, assuming positive)
# need to be able to handle multiple optimal alignments

import numpy as np

def alignment(seq1, seq2, match_score, mis_score, gap_pen, global_bool=True):
    if global_bool:
        grid, trace = global_align(seq1, seq2, match_score, mis_score, gap_pen)
    else:
        results = local_align(seq1, seq2, match_score, mis_score, gap_pen)
    
    opt_align = traceback(grid, trace, seq1, seq2)
    
    return opt_align, grid, trace

def match(i, j, seq1, seq2, match_score, mis_score):
    base_i = seq1[i-1]
    base_j = seq2[j-1]

    if base_i == base_j:
        return match_score
    else:
        return mis_score

def traceback(grid, trace, seq1, seq2):
    opt_align = []
    align1 = ''
    align2 = ''
    i, j = np.unravel_index(grid.argmax(), grid.shape)
    
    while i > 0 and j > 0:
        next_step = str(trace[i, j])
        if '1' in next_step:
            align1 = seq1[i-1] + align1
            align2 = '_' + align2
            i -= 1
        elif '2' in next_step:
            align1 = '_' + align1
            align2 = seq2[j-1] + align2
            j -= 1
        elif '3' in next_step:
            align1 = seq1[i-1] + align1
            align2 = seq2[j-1] + align2
            i -= 1
            j -= 1
    
    opt_align.append(align1)
    opt_align.append(align2)
    
    return opt_align
        
def global_align(seq1, seq2, match_score, mis_score, gap_pen):
    m = len(seq1) + 1
    n = len(seq2) + 1
    grid = np.zeros((m, n))
    trace = np.zeros((m, n))
    
    # rows are i and columns are j
    for i in range(m):
        grid[i][0] = gap_pen*-i
    for j in range(n):
        grid[0][j] = gap_pen*-j
    
    for i in range(1, m):
        for j in range(1, n):
            match_boost = match(i, j, seq1, seq2, match_score, mis_score)
            
            up = grid[i-1][j] - gap_pen
            left = grid[i][j-1] - gap_pen
            diag = grid[i-1][j-1] + match_boost
            
            high_score = max(up, left, diag)
            
            grid[i][j] = high_score
            
            trace_pointer = ''
            if up == high_score:
                trace_pointer += '1'
            if left == high_score:
                trace_pointer += '2'
            if diag == high_score:
                trace_pointer += '3'
            trace[i][j] = int(trace_pointer)

    return grid, trace
            
    
# def local_align(seq1, seq2, match_score, mis_score, gap_pen)

In [40]:
seq1 = 'AGATTAC'
seq2 = 'AGTTAC'

opt_align, grid, trace = alignment(seq1, seq2, match_score=1, mis_score=-1, gap_pen=3, global_bool=True)
opt_align

['AGATTAC', 'AG_TTAC']

# 4.  Statisics of Pairwise Alignments

Perform sequence alignment of the following sequences:

```
PAWHEAE
HEAGAWGHEE
```
Assume a match score of 1, a gap penalty of 3 and a substitution score of -1.  What is the alignment score?


Next, generate 100 random sequences with the same amino acid distribution as `PAWHEAE` and perform an alignment of each sequence to `HEAGAWGHEE`.   Calculate the Z-score.  What does the Z-score say about the significance of the alignment?   Are the scores normally distributed?  *You can use SciPy or any other modules to help.*

Repeat the process above, but now generate 1,000 and 10,000 random sequences and calculate the Z-score for each set.  Does the change in number of sequences alter your evaluation of the evolutionary relatedness of the sequences?

Considering the wall time for each run, comment of the feasibility for computing the Z-score significance while searching a large database of millions of sequences.

# 5.  Recent Approaches in Sequence Alignemnt

Search PubMed and identify a paper on a different sequence alignment method.  Do not use BLAST or FASTA.  Briefly (~1 paragraph) discuss what is different in these approaches and how they improve on Needleman-Wunch and/or Smith-Waterman.  You answer does not have to be uneccesarily technical, but please provide a concrete example of the improvement it makes.